In [1]:
from datasets import load_dataset,Dataset
START_SYM="<--start-->"
# use name="sample-10BT" to use the 10BT sample
# fw = load_dataset("/data/bert-chunker-v2/dataset/fineweb", name="sample/10BT", split="train", streaming=True)
# data_files = ["000_00000.parquet","001_00000.parquet","002_00000.parquet",]
data_files = ["para4000_cut.parquet",]

# fw = load_dataset("/data/bert-chunker-v2/dataset/fineweb/sample/10BT", data_files=data_files,split="train",num_proc=6,cache_dir='/data/bert-chunker-v2/dataset/cache')
# fw = load_dataset("/data/bert-chunker-v2/dataset/fineweb/sample/10BT",split="train",num_proc=6,cache_dir='/data/bert-chunker-v2/dataset/cache')
fw = load_dataset("/data/bert-chunker-v2/dataset/doubao_responses", data_files=data_files,split="train",num_proc=6,cache_dir='/data/bert-chunker-v2/dataset/cache')

print(fw)


/data/miniconda/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Setting num_proc from 6 back to 1 for the train split to disable multiprocessing as it only contains one shard.
Generating train split: 37538 examples [00:02, 17507.62 examples/s]

Dataset({
    features: ['text', 'id', 'dump', 'url', 'file_path', 'language', 'language_score', 'token_count', 'score', 'int_score', 'doubao', 'chunk', 'check'],
    num_rows: 37538
})


In [5]:

print(fw[1]['chunk'])



Taking Play Seriously
By ROBIN MARANTZ HENIG
Published: February 17, 2008
On a drizzly Tuesday night in late January, 200 people came out to hear a psychiatrist talk rhapsodically about play -- not just the intense, joyous play of children, but play for all people, at all ages, at all times. (All species too; the lecture featured touching photos of a polar bear and a husky engaging playfully at a snowy outpost in northern Canada.) Stuart Brown, president of the National Institute for Play, was speaking at the New York Public Library's main branch on 42nd Street. He created the institute in 1996, after more than 20 years of psychiatric practice and research persuaded him of the dangerous long-term consequences of play deprivation. In a sold-out talk at the library, he and Krista Tippett, host of the public-radio program ''Speaking of Faith,'' discussed the biological and spiritual underpinnings of play. Brown called play part of the ''developmental sequencing of becoming a human primate

In [3]:
d = fw.filter(lambda x: 'microcomputer' in x['input'],num_proc=6)
print(d)
print(d[2]['input'])

Filter (num_proc=6):   0%|          | 0/155598 [00:00<?, ? examples/s]

Filter (num_proc=6): 100%|██████████| 155598/155598 [00:00<00:00, 528482.63 examples/s]

Dataset({
    features: ['input'],
    num_rows: 10
})
Deeds is a tool, specifically designed for educational applications, for design and simulation of digital systems based on combinational and sequential blocks, finite state machines and microcomputers. Deeds is characterized by an extremely simple user interface and it is associated with a large repository of application projects, available on the web. The paper presents a newly developed feature of Deeds, conceived for introducing FPGA-based systems in a first course of digital design based on Deeds. This extension allows students to compile a project into an FPGA chip directly from Deeds, leaving in the background the operations performed by the FPGA specific EDA tool. The main advantage is a smoother transition between Deeds and a professional design environment.<--start--> Top Available Seniors at TOC Quarters
Tuesday night in Toms River the four sectional public school champs played a doubleheader in the Tournament of Champion

In [16]:
# d = fw.filter(lambda x: x["text"].count('\n')>=2 and x['token_count']<=150,num_proc=6)
# d = fw.filter(lambda x:  x['token_count']<=150 and x['token_count']>=50,num_proc=6)
d = fw.filter(lambda x:  x['token_count']>250,num_proc=6)

# def cut(example):
#     example["input"] = example['text']
#     return example

# # 使用 map 方法批量处理
# d = d.map(cut,num_proc=6)
# last_100_elements = d[0:10000]

# # 形成新的数据集
# d = Dataset.from_dict(last_100_elements)
# d.to_parquet("/data/bert-chunker-v2/dataset/fw-train-onepara.parquet")


Filter (num_proc=6): 100%|██████████| 3143000/3143000 [00:10<00:00, 311119.23 examples/s]


In [14]:
print(d)

Dataset({
    features: ['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'token_count'],
    num_rows: 29303
})


In [16]:
print(d[2]['text'])

print(d[2]['token_count'])
d = d.shuffle(seed=42)
print(d[2]['text'])

print(d[2]['token_count'])

Employment Lawyers £60,000 - £69,999 jobs in Manchester
Broaden your search
Refine your search
Found 3 Recruitment Consultant jobs
This Legal 500 accredited firm prides their success on providing pragmatic and quality legal advice to their wide range of clients nationwide. This we
This is a great opportunity to join a highly regarded and entrepreneurial Top 100 law firm.
Employment Solicitor - Tier 1 Manchester Firm
A well regarded Legal 500 ranked national law firm in Manchester is seeking an employment solicitor (claimant) at senior associate level to join. ...
119
|Topic Review (Newest First)|
|01-17-2007 10:03 AM|
Update on Canada's new sewer discharge laws
Some time back there was some chatter about Canada's intention of making their waters no discharge zones. It's my recollection that they were going to require a logbook of sorts that tracked inputs and outputs to the holding tank. The main issue was that there really aren't any pumpout facilities to speak of, at least around Des

In [17]:
print(d)

Dataset({
    features: ['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'token_count'],
    num_rows: 40651
})


In [18]:
def cut(example):
    START_SYM="<--start-->"
    example["input"] = example['text']+START_SYM
    return example

# 使用 map 方法批量处理
d = d.map(cut,num_proc=6)


Map (num_proc=6): 100%|██████████| 40651/40651 [00:01<00:00, 22457.69 examples/s]


In [19]:
d = d.remove_columns(['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'token_count'])

In [20]:
def merge_batch(batch, group_size=3):
    START_SYM="<--start-->"
    new_data = {"input": [],}
    for i in range(0, len(batch['input']), group_size):
        # 获取当前组的 4 条数据
        texts = batch['input'][i:i+group_size]
        
        # 合并数据
        merged_text = ' '.join(texts)
        merged_text = merged_text[:-len(START_SYM)]
        
        # 添加到新数据中
        new_data["input"].append(merged_text)
    return new_data

# 使用 Dataset.map 处理数据
new_dataset = d.map(
    merge_batch,
    batched=True,  # 启用批处理
    batch_size=1000,  # 每批次处理 1000 条原始数据
    remove_columns=d.column_names,  # 移除原始列
    num_proc=6,  # 使用 6 个进程并行处理
)

Map (num_proc=6):   0%|          | 0/40651 [00:00<?, ? examples/s]

Map (num_proc=6): 100%|██████████| 40651/40651 [00:00<00:00, 223073.78 examples/s]


In [23]:
print(new_dataset)

Dataset({
    features: ['input'],
    num_rows: 13578
})


In [24]:
# 获取最后 100 个元素
last_100_elements = new_dataset[-10000:]

# 形成新的数据集
d = Dataset.from_dict(last_100_elements)
print(d)

Dataset({
    features: ['input'],
    num_rows: 10000
})


In [25]:
d.to_parquet("/data/bert-chunker-v2/dataset/fw-test-cross.parquet")

Creating parquet from Arrow format: 100%|██████████| 10/10 [00:00<00:00, 101.35ba/s]


14685418